## Libraries

In [1]:
import tempfile
import math
import os
from subprocess import call
import matplotlib.pyplot as plt
from f90nml import *
from array import *
import numpy as np
import pandas as pd 
import seaborn as sns
from statistics import *
import shutil
import sys
import time
from numpy import mean
from numpy import std
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import linregress

#read in printFolder, makeDefNameList, newFolder, deleteFolder, readOutput from coreFunctions.py
from coreFunctions import *

#read in plotting functionality from plottingFunctions.py
from plottingFunctions import *

call("rm -rf tmp*",shell=True)

dfModel = []#will contain output from our model

## Run Model

In [13]:
start = time.time()
nameList = makeDefNamelist() #make namelist
#-------------------------------------Boolean Inputs--------------------------------------------------
plot=False;#make the fancy plots?
save=False#save the plots
coupled=False; #if this is False, program quits once EBM reaches equilibrium
analyze = False; #whether or not to analyze the population parts of model
lverbose=False; #controls terminal output, used for debugging
showInputs = False; #prints the namelist before running the program
#-------------------------------------Other Important Inputs-------------------------------------------------# maxPopList = [10000,40000,70000,100000] #specify all maxpops to use
exp =0
runTime=500                          #Change runtime           (years)
maxPopList = [10000]
#----------------------------------Set NameList Values to Specified Inputs---------------------
nameList['ebm']['lverbose'] = lverbose
def pco2Finder(goalEqTemp, distance, lverbose):
    distList = [distance]
    maxPopList = [10000]
    currEqTemp = 0
    pco20    = 10.3
    nameList['ebm']['pco20'] = pco20*10**-6
    goalPco2 = 0
    plot = False
    save=False
    analyze=False
    popDeath = []
    dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 500, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
    if(eqTemp[0]>= goalEqTemp):
        return np.nan
 #   print(f"Equilbrium Reached at temp: {eqTemp:.2f}, and time: {eqTime:.0f}","\n")
    while(True):
        call("rm -rf tmp*",shell=True)
        if(currEqTemp >= goalEqTemp):
            goalPco2 = pco20
            break
        if(goalEqTemp-currEqTemp > 50):
            pco20 *= 1.5 #increment pco2 by 1% of its value
            nameList['ebm']['pco20'] = pco20*10**-6
            dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 500, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
            if np.isnan(eqTemp[0]): return np.nan
            currEqTemp = eqTemp[0]
            if lverbose: print(f"pCO20:{pco20:.3f},  Equilbrium Reached at temp: {eqTemp[0]:.2f}, and time: {eqTime[0]:.0f},  goalTemp: {goalEqTemp:.0f}","\n")
        if(goalEqTemp-currEqTemp <= 50):
            pco20 += 0.1*pco20
            nameList['ebm']['pco20'] = pco20*10**-6
            dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 3000, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
            if np.isnan(eqTemp[0]): return np.nan
            currEqTemp = eqTemp[0]
            if lverbose: print(f"pCO20:{pco20:.3f},  Equilbrium Reached at temp: {eqTemp[0]:.2f}, and time: {eqTime[0]:.0f},  goalTemp: {goalEqTemp:.0f}","\n")
    return round(goalPco2,3)
def dTdPFinder(goalPco2, distance):
    if not np.isnan(goalPco2):
        distList  = [distance]
        #find mean
        nameList['ebm']['pco20'] = goalPco2*10**-6
        dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 500, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
        eqTempMean = eqTemp[0]
        #find upper bound
        pco20Plus = goalPco2 + 0.1*goalPco2 #increment pco2 by 1% of its value
        nameList['ebm']['pco20'] = pco20Plus*10**-6
        dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 500, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
        eqTempPlus = eqTemp[0]
        #find lower bound
        pco20Minus = goalPco2 - 0.1*goalPco2 #increment pco2 by 1% of its value
        nameList['ebm']['pco20'] = pco20Minus*10**-6
        dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, False, 500, plot, save, analyze,"driver.exe",maxPopList,distList,showInputs, experiment=exp, scaleInitPop=True)
        eqTempMinus = eqTemp[0]
        m1 = (eqTempMean - eqTempMinus)/(goalPco2 - pco20Minus)
        m2 = (eqTempPlus - eqTempMean)/(pco20Plus - goalPco2)
        m3 = (eqTempPlus - eqTempMinus)/(pco20Plus - pco20Minus)
#        print(f"m1: {m1:.3e}, m2: {m2:.3e}, m3: {m3:.3e}")
        return (m1+m2)/2
    else:
        return np.nan
#------------------------------------Run the Model----------------------------------------------------
n = 5
distanceList = [] #x
tempList = [] #y
dTdPList = []
dimVarList = []
pco2List = []
popDeathList = []
distances = np.linspace(0.934, 1.147, n)
goalEqTemps = np.linspace(273.15,373.15, n)
numRuns = len(distances)*len(goalEqTemps)
counter = 0

for dist in distances:
    end = time.time()
    timeMinutes = (end - start)/60
    print(f"Runtime: {timeMinutes:.2f} minutes have passed")
    print("\n", "Distance: ", round(dist,3), " AU")
    for temp in goalEqTemps:
        distanceList.append(dist)
        tempList.append(temp)
        goalPco2 = pco2Finder(temp, dist, False)#False = suppress output
        if np.isnan(goalPco2): popDeathList.append(np.nan)
        else:
            nameList['ebm']['pco20'] = goalPco2*10**-6
            dfModel,dfData,eq, eqTemp, eqTime, popDeath = runModel(nameList, True, 3000, False, False, True,"driver.exe",[10000],[dist],showInputs, experiment=0, scaleInitPop=True)
            popDeathList.append(popDeath[0])
        pco2List.append(goalPco2)
  #      popDeathList.append(popDeath)
#        print(f"pCO2: {goalPco2}")
        goaldTdP = dTdPFinder(goalPco2, dist)
        dTdPList.append(goaldTdP)
#        print(f"dTdP: {goaldTdP:.3e}")
        if not np.isnan(goalPco2): dimVar=(maxPopList[0]*nameList['ebm']['rco2']*goaldTdP)/(nameList['ebm']['rBirth0']*nameList['ebm']['dtemp'])
        if np.isnan(goalPco2): dimVar = np.nan
        dimVarList.append(dimVar)
        print(str(round((counter/numRuns)*100,2)) + "% Completion" )
        counter += 1
#        print(f"Dimensionless Variable: {dimVar}","\n")
#------------------------------------Plot the Results----------------------------------------------
dfTemp = pd.read_csv('anomaly.csv');#data from NASA GISS: https://data.giss.nasa.gov/gistemp/graphs_v4/
dfPopCo2 = pd.read_csv('world_stats.csv');#data from Frank, Adam, and Woodruff Sullivan.
#compareModelOutput(dfModel,dfTemp,dfPopCo2,eqTime)#compare the models output to true data
end = time.time()
timeMinutes = (end - start)/60
print(f"Runtime: {timeMinutes:.2f} minutes")
call("echo End of Python Notebook Reached", shell=True);

Runtime: 0.00 minutes have passed

 Distance:  0.934  AU
[nan]
0.0% Completion
[nan, nan]
4.0% Completion
[nan, nan, nan]
8.0% Completion
[0]
[nan, nan, nan, 0.8955471086]
Equilibrium was not reached

12.0% Completion
Equilibrium was not reached

[nan, nan, nan, 0.8955471086, nan]
16.0% Completion
Runtime: 1.15 minutes have passed

 Distance:  0.987  AU
[nan, nan, nan, 0.8955471086, nan, nan]
20.0% Completion
[0]
[nan, nan, nan, 0.8955471086, nan, nan, 228.9176306]
24.0% Completion
[0]
[nan, nan, nan, 0.8955471086, nan, nan, 228.9176306, 4.653140719]


KeyboardInterrupt: 

## Plotting

In [ ]:
#fig = plt.figure(figsize=(17,7))
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("prodData.csv")
# df2 = df[df.logGamma.isna() & df["dist"]<1.0]
# #print(print(df2[df2['temp'].notna()]))
# ind=np.asarray(df.index.difference(df2.index))
# for i in in d:
#     print(df.iloc[i])
# df["dist"] = np.round(distanceList,3) #x
# df["temp"] = np.round(tempList)     #y
# df["logdTdP"] = np.round(np.log10(dTdPList))
# df["logpco20"] = np.log10(pco2List)
# df["gamma"] = dimVarList
# df["logGamma"] = np.log10(dimVarList)
# df["tFreeFall"] = -np.log10(dimVarList)

# A=0.04;#birth ratee
# tFreeFall = lambda g: min(1, 1/g)
# df["tFreeFall"] = df["gamma"].apply(tFreeFall)
# df["tGen"] = df["tFreeFall"]/A
#print(df[df["logpco20"].isna()])

df2 = df[["dist", "temp", "logpco20"]]
df2 = df2.pivot("dist", "temp", "logpco20")

colorMap = "plasma_r"
x = np.asarray(df2.index)
y = np.asarray(df2.columns)
xx, yy = np.meshgrid(x,y)
zz = np.zeros_like(xx)
for i in range(zz.shape[0]):
    for j in range(zz.shape[1]):
        zz[j, i] = df2.iloc[i,j]
            
df3 = df[["dist", "temp", "logGamma"]]
df3 = df3.pivot("dist", "temp", "logGamma")
colorMap = "viridis"
x1 = np.asarray(df3.index)
y1 = np.asarray(df3.columns)
xx1, yy1 = np.meshgrid(x1,y1)
zz1 = np.zeros_like(xx1)
for i in range(zz1.shape[0]):
    for j in range(zz1.shape[1]):
        zz1[j,i] = df3.iloc[i,j]

# contourLines = np.asarray([-4,-3,-2,-1,0])
# fig, ax = plt.subplots(figsize=(10,7.5))
# fig.subplots_adjust(top=.8)
# cs1 = ax.pcolormesh(yy1, xx1, zz1, alpha=0.7,cmap=colorMap)
# cs2 = ax.contour(yy1, xx1, zz1, cmap=colorMap, levels=contourLines)
# #palette = plt.cm.jet
# #palette.set_bad ('w',0.3) # Bad values (i.e., masked, set to grey 0.8 
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}$") #set contour labels
# ax.set(xlabel=r'$Temperature\ (K)$', ylabel=r'$Distance\ (AU)$', title=r"Distance  (AU)  vs  Temperature  (K)  vs  $logGamma\ (log\ \gamma)$") #set axis labels;
# cbar = fig.colorbar(cs1, ax=ax);
# cbar.set_label(r"$logGamma\ (log\ \gamma)$")
#plt.savefig("summaryFig2.png")

In [ ]:
#fig = plt.figure(figsize=(17,7))
# import warnings
# warnings.filterwarnings("ignore")

# df = pd.read_csv("prodData.csv")

# A=0.04;#birth ratee
# tFreeFall = lambda g: max(1, 1/g)
# df["tFreeFall"] = df["gamma"].apply(tFreeFall)
# df["tGen"] = df["tFreeFall"]/A
# #print(df[df["logpco20"].isna()])

# df2 = df[["logGamma", "logdTdP", "tGen"]]
# # df2 = df2.drop_duplicates().reset_index()
# # print(df2)
# df2 = df2.pivot("logGamma", "logdTdP", "tGen")
# colorMap = "plasma_r"
# x = np.asarray(df2.index)
# y = np.asarray(df2.columns)
# xx, yy = np.meshgrid(x,y)
# zz = np.zeros_like(xx)
# for i in range(20):
#     for j in range(zz.shape[1]-2):
#         zz[j, i] = df2.iloc[i,j]
            

# # contourLines = np.asarray([-4,-3,-2,-1,0])
# fig, ax = plt.subplots(figsize=(10,7.5))
# fig.subplots_adjust(top=.8)
# cs1 = ax.pcolormesh(yy, xx, zz, alpha=0.7,cmap=colorMap)
# cs2 = ax.contour(yy, xx, zz, cmap=colorMap, levels=contourLines)
# #palette = plt.cm.jet
# #palette.set_bad ('w',0.3) # Bad values (i.e., masked, set to grey 0.8 
# #ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%d}$") #set contour labels
# ax.clabel(cs2, inline=True, fontsize=14, colors="Black", fmt=r"$10^{%1d}$") #set contour labels
# ax.set(xlabel=r'$dTdP\ (K/ppm)$', ylabel=r'$Gamma\ (\gamma)$', title=r"dTdP  (K/ppm)  vs  Gamma  ($\gamma$)  vs  Collapse Timescale $(t_{years})$") #set axis labels;
# cbar = fig.colorbar(cs1, ax=ax);
# cbar.set_label(r"$logGamma\ (log\ \gamma)$")
#plt.savefig("summaryFig3.png")

In [ ]:
# df = pd.DataFrame()
# df["dist"] = np.round(distanceList,3) #x
# df["temp"] = np.round(tempList)     #y
# df["dTdP"] = np.round(np.log10(dTdPList))
# df["pco20"] = np.log10(pco2List)
# df["gamma"] = np.log10(dimVarList)

# fig, axes = plt.subplots(figsize=(15,5), nrows=1, ncols=2)
# cmap_ = "YlGnBu"

# ax = axes[0]
# df2 = df.pivot("dist", "temp", "gamma")
# sns.heatmap(df2, ax=ax, cmap = "Greens",  annot=True)
# ax.set_ylabel("Distance (AU)")
# ax.set_xlabel("Temperature (K)")
# ax.set_title("Distance vs Temperature vs LogGamma")

# ax = axes[1]
# df3 = df.pivot("dist", "temp", "pco20")
# sns.heatmap(df3, ax=ax, cmap = "Purples", linewidths=.2)
# ax.set_ylabel("Distance (AU)")
# ax.set_xlabel("Temperature (K)")
# ax.set_title("Distance vs Temperature vs LogpCO20")


# plt.tight_layout()
# plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(15,5))
contourLines = np.linspace(-90, -15, 4)

dfNew = pd.DataFrame()
x, y = np.linspace(-10,5,5), np.linspace(-10,10,5)
z = lambda x, y: y**2

xList = []
yList = []
zList = []
for i in x:
    for j in y:
        xList.append(i)
        yList.append(j)
        zList.append(z(i,j))
dfNew["x"] = xList
dfNew["y"] = yList
dfNew["z"] = zList
dfNew = dfNew.pivot("x", "y", "z")

#ax = fig.add_subplot(1 , 3, 1)
ax = plt.gca()
sns.heatmap(dfNew, ax=ax,annot=True)
plt.show()
#ax.set(xticks=[], yticks=[])

xx, yy = np.meshgrid(x,y)
zz = np.zeros_like(xx)
for i in range(zz.shape[0]):
    for j in range(zz.shape[1]):
        zz[j,i] = dfNew.iloc[i,j]

# ax = fig.add_subplot(1 , 3, 2)
# cs = ax.contour(xx, yy, zz, levels=contourLines, cmap='coolwarm')
# ax.clabel(cs, inline=True, fontsize=12)
# ax.set(xlabel=r'$X$', ylabel=r'$Y$')
# fig.tight_layout()


# ax = fig.add_subplot(1 , 3, 3, projection='3d')
# ax.plot_surface(xx, yy, zz, cmap='coolwarm', alpha=0.5)
# for level, contour in zip(contourLines, cs.allsegs):
#     dat0 = contour[0]
#     ax.plot(dat0[:, 0], dat0[:, 1], level, color='k')
#     ax.text(dat0[0, 0], dat0[0, 1], level, '{0}'.format(level), zdir=(1, 0, 0), ha='center', va='top')
# ax.view_init(elev=20, azim=-45)
# ax.set(xlabel=r'$X$', ylabel=r'$Y$', zlabel=r'$Z$', title=r'$Z = -(X^2 + Y^2)$')
# plt.tight_layout()
# plt.show()
fig = plt.figure()
ax = plt.gca()
cs = plt.contourf(yy, xx, zz, alpha=.5)
cs2 = plt.contour(yy, xx, zz)
ax.clabel(cs2)
fig.colorbar(cs)
plt.show()
# plt.pcolormesh(xx,yy,zz)